In [1]:
# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#     return false;
# }

In [2]:
%reload_ext autoreload
%autoreload 1
from IPython.display import display, clear_output

import pymongo

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from viewing import *
from util import *
%aimport sample
from sample import Sample

In [3]:
# Configuration of data frames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [4]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['thesis']
db_data = database['data']

In [5]:
# To obtain this data from the database

# boards_list = [b['_id'] for b in db_data.aggregate([{"$group": { "_id": "$Board"} }])]
# boards_32 = [b['_id'] for b in db_data.aggregate([{"$match": { "Type": {"$eq": 32} } }, {"$group": { "_id": "$Board" } }])]
# boards_64 = np.setdiff1d(boards_list, boards_32, assume_unique=False).tolist()

boards_32 = open('csv/boards_32.txt', 'r').read().splitlines()[0].split(',')
boards_64 = open('csv/boards_64.txt', 'r').read().splitlines()[0].split(',')
boards_list = boards_32 + boards_64

# Obtención de las métricas

## Uniformity

`Uniformity` es la relación entre el número de '0' y de '1' que hay en una muestra de datos. El valor ideal está en torno a __50%__. Para calcularla, sumaremos todos los '1' que hay en cada muestra y dividiremos entre el numero de bits que hay por muestra.

Matematicamente, se representa por la siguient formula:

$$\text{Uniformity} = \frac{1}{n} \sum\limits_{l=1}^n r_{i,l} \times 100 \%$$

Donde $r_{i, l}$ representa el bit numero _l_ de la muestra y $n$ el número de placas. Nuestras muestras son de 4096 bits y disponemos de 206 placas.

---

Como se puede ver en los resultados, la uniformidad de las muestras está cercana al valor ideal.
Los valores que desvian la media son las primeras muestras de cada placa, esto es, la primera muestra que se toma de la placa una vez se ha programado. Esto es asi para todas las regiones

In [7]:
uniformity_df = pd.DataFrame(columns=['_id', 'Board', 'Type', 'Wafer', 'Lot', 'X', 'Y',
                                      'Timestamp', 'Day', 'Month', 'Year', 'H', 'M', 'S',
                                      'Temp', 'Vdd', 'Region', 'Uniformity'])

for i, dump in enumerate(db_data.find()):
    d = sample.Sample(dump)
    uniformity_df = uniformity_df.append({'_id': d._id, 'Board': d.Board, 'Type': d.Type, 'Wafer': d.Wafer, 'Lot': d.Lot, 'X': d.X, 'Y': d.Y,
                                          'Timestamp': d.Timestamp, 
                                          'Day': d.Datetime.day, 'Month': d.Datetime.month, 'Year': d.Datetime.year,
                                          'H': d.Datetime.hour, 'M': d.Datetime.minute, 'S': d.Datetime.second,
                                          'Temp': d.Temp, 'Vdd': d.Vdd,
                                          'Region': d.Mem_pos, 'Uniformity': d.uniformity()}, ignore_index=True)
    
    if i % 5_000 == 0:
        print(f'\rCalculated {i} samples')

Calculated 0 samples
Calculated 5000 samples
Calculated 10000 samples
Calculated 15000 samples
Calculated 20000 samples
Calculated 25000 samples
Calculated 30000 samples
Calculated 35000 samples
Calculated 40000 samples
Calculated 45000 samples
Calculated 50000 samples
Calculated 55000 samples
Calculated 60000 samples
Calculated 65000 samples
Calculated 70000 samples
Calculated 75000 samples
Calculated 80000 samples
Calculated 85000 samples
Calculated 90000 samples
Calculated 95000 samples
Calculated 100000 samples
Calculated 105000 samples
Calculated 110000 samples
Calculated 115000 samples
Calculated 120000 samples
Calculated 125000 samples
Calculated 130000 samples
Calculated 135000 samples
Calculated 140000 samples
Calculated 145000 samples
Calculated 150000 samples
Calculated 155000 samples
Calculated 160000 samples
Calculated 165000 samples
Calculated 170000 samples
Calculated 175000 samples
Calculated 180000 samples
Calculated 185000 samples
Calculated 190000 samples
Calculated 

In [ ]:
uniformity_df.to_csv('./csv/uniformity.csv', index=False)

In [27]:
uniformity_df[uniformity_df['Type'] == 32].describe()

,Temp,Vdd,Uniformity
count,13983.000000,13983.000000,13983.000000
mean,568.181561,1364.958900,43.101197
std,255.463444,655.466211,10.347920
min,32.696629,3.211176,5.273438
25%,684.000000,1669.000000,41.699219
50%,689.000000,1677.000000,47.729492
75%,695.000000,1685.000000,48.706055
max,713.000000,1716.000000,51.440430


In [26]:
uniformity_df[uniformity_df['Type'] == 64].describe()

,Temp,Vdd,Uniformity
count,2.292300e+05,229230.000000,229230.000000
mean,-1.158442e+07,3.200201,47.410588
std,1.248964e+08,0.528076,8.099382
min,-9.264558e+08,-0.000111,4.370117
25%,2.695652e+01,3.257168,48.803711
50%,2.868852e+01,3.288179,49.511719
75%,3.043011e+01,3.301964,50.122070
max,8.013122e+08,8.142920,53.222656


In [46]:
summary_columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
uniformity_global_summary_df = pd.DataFrame(columns=['Region'] + summary_columns)
summaries_dict = {}

# Generate a summary for every type of board, that is 32 and 64
for Type in [32, 64]:
    uniformity_type_df = uniformity_df[uniformity_df['Type'] == Type]
    uniformity_grouped_by_regions_df = uniformity_type_df.groupby('Region')
    # Data frame to store the board type summary
    uniformity_temp_df = pd.DataFrame(columns=['Region'] + summary_columns)
    
    for mem_region, region_group_df in uniformity_grouped_by_regions_df:
        summary_df = region_group_df.describe()
        summary_data = {}
        summary_data['Region'] = mem_region
        
        for value in summary_columns:
            summary_data[value] = summary_df.loc[value][2]
            
        uniformity_temp_df = uniformity_temp_df.append(summary_data, ignore_index=True)

    # Append to the list fo data frames
    summaries_dict[str(Type)] = uniformity_temp_df

### Board 32

In [47]:
summaries_dict['32']

,Region,count,mean,std,min,25%,50%,75%,max
0,0x20000000,444.0,5.596044,0.078455,5.273438,5.541992,5.590820,5.639648,5.834961
1,0x20000200,443.0,14.205182,1.300559,10.205078,13.964844,13.964844,13.964844,21.093750
2,0x20000400,440.0,31.391491,2.808765,17.675781,32.080078,32.080078,32.080078,36.206055
3,0x20000600,438.0,39.497605,1.471370,29.296875,39.013672,39.526367,39.892578,42.456055
4,0x20000800,437.0,47.168918,2.575128,42.456055,46.679688,48.095703,48.828125,50.756836
5,0x20000a00,437.0,46.877514,2.709769,41.870117,46.655273,47.802734,48.779297,50.708008
6,0x20000c00,437.0,47.698877,1.896332,44.482422,46.850586,48.339844,49.047852,51.000977
7,0x20000e00,437.0,47.561387,1.797542,44.433594,46.630859,48.266602,48.828125,50.122070
8,0x20001000,437.0,46.802093,2.840975,41.650391,45.874023,48.046875,48.852539,50.708008
9,0x20001200,437.0,47.320710,2.621489,42.578125,47.021484,48.413086,49.218750,51.123047


### Board 64

In [48]:
summaries_dict['64']

,Region,count,mean,std,min,25%,50%,75%,max
0,0x20000000,3610.0,4.687466,0.324714,4.370117,4.565430,4.638672,4.711914,6.689453
1,0x20000200,3599.0,13.741033,0.594567,10.351562,13.720703,13.720703,13.720703,19.433594
2,0x20000400,3597.0,32.105531,1.343966,19.091797,32.177734,32.177734,32.177734,44.213867
3,0x20000600,3593.0,39.684349,1.264967,21.972656,39.355469,39.697266,40.039062,44.018555
4,0x20000800,3593.0,49.378648,1.731594,33.593750,49.023438,49.584961,50.146484,52.270508
5,0x20000a00,3592.0,49.428975,1.471625,41.040039,48.901367,49.560547,50.195312,52.319336
6,0x20000c00,3589.0,49.463353,1.451779,41.918945,49.121094,49.707031,50.195312,52.807617
7,0x20000e00,3590.0,49.522933,1.187415,43.115234,49.047852,49.633789,50.170898,52.124023
8,0x20001000,3587.0,49.390166,1.335284,40.844727,48.950195,49.560547,50.122070,52.001953
9,0x20001200,3586.0,49.485712,1.550577,41.552734,48.974609,49.658203,50.292969,52.587891


In [49]:
summaries_dict['32'].to_csv('./csv/uniformity_summary_32.csv', index=False)
summaries_dict['64'].to_csv('./csv/uniformity_summary_64.csv', index=False)

## Bit-aliasing

The design and operation conditions can affect the frequency of the RO responses. As a
240 consequence, some RO responses might give a fixed value (e.g., “00” or “01”) in all devices. To measure
241 this undesirable behaviour the bit-aliasing metric. Mathematically, it is described as follow

## Reliability

## Uniqueness

## 